In [1]:
%load_ext autoreload
%autoreload 2

from util import *
from b_algorithm import algorithm as b1
from myalgorithm import algorithm as a1

In [3]:
import os
import traceback

prob_dir = "alg_test_problems"

problem_list = os.listdir(prob_dir)

timelimit = 60

record = []

for file_name in problem_list:

    problem_file = os.path.join(prob_dir, file_name)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    alg_start_time = time.time()

    exception = None

    solution = None
    try:
        # Run algorithm!
        solution, bundles, weight = a1(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
    except Exception as e:
        exception = f'{e}'
        print(traceback.format_exc())
        break


    alg_end_time = time.time()

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)
    
    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    checked_solution['time'] = alg_end_time - alg_start_time
    checked_solution['timelimit_exception'] = (alg_end_time - alg_start_time) > timelimit + 1 # allowing additional 1 second!
    checked_solution['exception'] = exception

    checked_solution['prob_name'] = prob['name']
    checked_solution['prob_file'] = problem_file

    record.append(checked_solution)


import csv
with open("result_cur.csv", 'w', newline = '') as f:
    writer = csv.writer(f)

    writer.writerow(["prob","avg_cost","feasible","time"])

    for row in record:
        writer.writerow([row['prob_name'], row['avg_cost'], row['feasible'], row['time']])


Length of BIKE-bundle 1-orders: 100
Length of BIKE-bundle 2-orders: 2200
Length of BIKE-bundle 3-orders: 12552
Length of BIKE-bundle 4-orders: 2698
Length of BIKE-bundle 5-orders: 0
Length of WALK-bundle 1-orders: 61
Length of WALK-bundle 2-orders: 6
Length of WALK-bundle 3-orders: 0
Length of WALK-bundle 4-orders: 0
Length of WALK-bundle 5-orders: 0
Length of CAR-bundle 1-orders: 100
Length of CAR-bundle 2-orders: 1378
Length of CAR-bundle 3-orders: 2208
Length of CAR-bundle 4-orders: 68
Length of CAR-bundle 5-orders: 0
Length of BIKE-bundle 1-orders: 100
Length of BIKE-bundle 2-orders: 2132
Length of BIKE-bundle 3-orders: 10026
Length of BIKE-bundle 4-orders: 2799
Length of BIKE-bundle 5-orders: 0
Length of WALK-bundle 1-orders: 66
Length of WALK-bundle 2-orders: 10
Length of WALK-bundle 3-orders: 0
Length of WALK-bundle 4-orders: 0
Length of WALK-bundle 5-orders: 0
Length of CAR-bundle 1-orders: 100
Length of CAR-bundle 2-orders: 1222
Length of CAR-bundle 3-orders: 1728
Length of CA

In [14]:
import os
import traceback

prob_dir = "alg_test_problems"

problem_list = os.listdir(prob_dir)

timelimit = 60

record = []

for file_name in problem_list:

    problem_file = os.path.join(prob_dir, file_name)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    alg_start_time = time.time()

    exception = None

    solution = None
    try:
        # Run algorithm!
        solution = b1(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
    except Exception as e:
        exception = f'{e}'
        print(traceback.format_exc())
        break


    alg_end_time = time.time()

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)
    
    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    checked_solution['time'] = alg_end_time - alg_start_time
    checked_solution['timelimit_exception'] = (alg_end_time - alg_start_time) > timelimit + 1 # allowing additional 1 second!
    checked_solution['exception'] = exception

    checked_solution['prob_name'] = prob['name']
    checked_solution['prob_file'] = problem_file

    record.append(checked_solution)


import csv
with open("result_past.csv", 'w', newline = '') as f:
    writer = csv.writer(f)

    writer.writerow(["prob","avg_cost","feasible","time"])

    for row in record:
        writer.writerow([row['prob_name'], row['avg_cost'], row['feasible'], row['time']])


KeyboardInterrupt: 